In [1]:
import random
import urlparse
import numpy as np
import joblib as jl
import xgboost as xgb

import extract
import get_rank
import get_cert
import get_whois
import get_pagerank
import get_ip
import get_sus_tld
import make_feature
from file_io import load_urls

In [2]:
total_mal_urls = load_urls("../std_data/mal.csv")
total_ben_urls = load_urls("../std_data/ben.csv")
total_unk_urls = load_urls("../std_data/unk.csv")
total_urls = set(total_ben_urls) | set(total_mal_urls) | set(total_unk_urls)
total_domains = list(set([urlparse.urlparse(_).hostname for _ in total_urls]))
print len(total_urls)
print len(total_domains)

2018-06-20 16:48:28 file_io.py [line:52] DEBUG	URLs Count:	14185
2018-06-20 16:48:28 file_io.py [line:52] DEBUG	URLs Count:	61595
2018-06-20 16:48:28 file_io.py [line:52] DEBUG	URLs Count:	1473


77225
9036


In [9]:
# tst_url_ip_map      = jl.load('../data/exp_rule/tst/url_ip_map.jl.z')
# tst_url_ip_dmap     = jl.load('../data/exp_rule/tst/url_ip_dmap.jl.z')
# tst_domain_rank_map = jl.load('../data/exp_rule/tst/domain_rank_map.jl.z')
# tst_domain_cert_map = jl.load('../data/exp_rule/tst/domain_cert_map.jl.z')

url_ip_map      = jl.load("../data/exp_rule/ip/url_ip_map2.jl.z" )
url_ip_dmap     = jl.load('../data/exp_rule/ip/url_ip_dmap2.jl.z')
domain_rank_map = jl.load('../data/exp_rule/alexa_rank/alexa_rank2.jl.z')
domain_cert_map = jl.load('../data/exp_rule/cert/certificate2.jl.z')

# url_ip_map.update(tst_url_ip_map)
# url_ip_dmap.update(tst_url_ip_dmap)
# domain_cert_map.update(tst_domain_cert_map)
# domain_rank_map.update(tst_domain_rank_map)

# jl.dump(url_ip_map, "../data/exp_rule/ip/url_ip_map2.jl.z")
# jl.dump(url_ip_dmap, '../data/exp_rule/ip/url_ip_dmap2.jl.z')
# jl.dump(domain_cert_map, '../data/exp_rule/cert/certificate2.jl.z')
# jl.dump(domain_rank_map, '../data/exp_rule/alexa_rank/alexa_rank2.jl.z')

domain_whois_map = get_whois.load_whois_dict(path = "../data/exp_rule/whois")
domain_pr_map = jl.load("../data/exp_rule/pr/pr.jl.z")
sus_domain = jl.load("../data/exp_rule/sus_domain/sus_domain.jl.z")

In [10]:
print len(url_ip_map)
print len(url_ip_dmap)
print len(domain_cert_map)
print len(domain_rank_map)
print ""
print len(domain_whois_map)
print len(domain_pr_map)
print len(sus_domain)

77225
12393
9036
9036

3348
9036
262


In [20]:
split_ratio = 0.6
train_ben_urls = random.sample(total_ben_urls, int(len(total_ben_urls)*split_ratio))
train_mal_urls = random.sample(total_mal_urls, int(len(total_mal_urls)*split_ratio))

test_ben_urls = list(set(total_ben_urls) - set(train_ben_urls))
test_mal_urls = list(set(total_mal_urls) - set(train_mal_urls))

print len(total_ben_urls), len(train_ben_urls), len(test_ben_urls)
print len(total_mal_urls), len(train_mal_urls), len(test_mal_urls)

61595 36957 24638
14185 8511 5674


In [21]:
data_obj = extract.FeatureExtract(train_mal_urls, train_ben_urls)
data_obj.feature_extract(url_ip_dmap)

In [22]:
print "ben filename", len(data_obj.ben_filename)
print "mal filename", len(data_obj.mal_filename)
print "ben filepath", len(data_obj.ben_filepath)
print "mal filepath", len(data_obj.mal_filepath)

print "mal pdomain tokens", len(data_obj.mal_pdomain_tokens)
print "mal sdomain tokens", len(data_obj.mal_sdomain_tokens)
print "ben pdomain tokens", len(data_obj.ben_pdomain_tokens)
print "ben sdomain tokens", len(data_obj.ben_sdomain_tokens)

print "ben ips", len(data_obj.mal_ips)
print "ana path", len(data_obj.domain_path_map)
print "ana port", len(data_obj.domain_port_map)

ben filename 18662
mal filename 5655
ben filepath 5405
mal filepath 518
mal pdomain tokens 116
mal sdomain tokens 79
ben pdomain tokens 627
ben sdomain tokens 573
ben ips 36
ana path 6134
ana port 91


In [23]:
feat_obj = make_feature.build_feature(data_obj, url_ip_map, url_ip_dmap, domain_whois_map, domain_rank_map, domain_cert_map, \
                                      domain_pr_map, sus_domain)

In [24]:
train_mal_x = feat_obj.build_feature_set(train_mal_urls)
train_ben_x = feat_obj.build_feature_set(train_ben_urls)
test_mal_x = feat_obj.build_feature_set(test_mal_urls)
test_ben_x = feat_obj.build_feature_set(test_ben_urls)
unk_x = feat_obj.build_feature_set(total_unk_urls)

In [25]:
train_x = np.concatenate((train_mal_x,train_ben_x), axis = 0)
train_y = [1] * len(train_mal_x) + [0] * len(train_ben_x)

test_x =  np.concatenate((test_mal_x, test_ben_x), axis = 0)
test_y = [1] * len(test_mal_x) + [0] * len(test_ben_x)

In [26]:
dtrain = xgb.DMatrix(train_x, train_y)
dtest  = xgb.DMatrix(test_x, test_y)

params={
'booster': 'gbtree',
'objective':'binary:logistic',
'gamma':0.1,  
'max_depth':12, 
'lambda':2,  
'subsample':1, 
'colsample_bytree':1, 
'min_child_weight':1, 
'silent':0 ,
'eta': 0.2, 
'seed':1000,
'nthread':8
}

watchlist  = [(dtest,'eval'), (dtrain,'train')]  
num_round = 200
bst = xgb.train(params, dtrain, num_round, watchlist) 

[0]	eval-error:0.010887	train-error:0.002265
[1]	eval-error:0.010887	train-error:0.002265
[2]	eval-error:0.010986	train-error:0.002265
[3]	eval-error:0.010986	train-error:0.002265
[4]	eval-error:0.010986	train-error:0.002265
[5]	eval-error:0.010953	train-error:0.002155
[6]	eval-error:0.010722	train-error:0.001869
[7]	eval-error:0.010689	train-error:0.001825
[8]	eval-error:0.010722	train-error:0.001781
[9]	eval-error:0.010788	train-error:0.001759
[10]	eval-error:0.010986	train-error:0.001759
[11]	eval-error:0.010821	train-error:0.00165
[12]	eval-error:0.010986	train-error:0.00165
[13]	eval-error:0.010788	train-error:0.001606
[14]	eval-error:0.010953	train-error:0.001584
[15]	eval-error:0.010953	train-error:0.001584
[16]	eval-error:0.011052	train-error:0.001562
[17]	eval-error:0.011052	train-error:0.001518
[18]	eval-error:0.010986	train-error:0.001452
[19]	eval-error:0.01092	train-error:0.001386
[20]	eval-error:0.010953	train-error:0.001364
[21]	eval-error:0.010986	train-error:0.00132
[2

[180]	eval-error:0.0128	train-error:0.000506
[181]	eval-error:0.0128	train-error:0.000506
[182]	eval-error:0.0128	train-error:0.000506
[183]	eval-error:0.0128	train-error:0.000506
[184]	eval-error:0.0128	train-error:0.000506
[185]	eval-error:0.0128	train-error:0.000506
[186]	eval-error:0.0128	train-error:0.000506
[187]	eval-error:0.0128	train-error:0.000506
[188]	eval-error:0.0128	train-error:0.000506
[189]	eval-error:0.0128	train-error:0.000506
[190]	eval-error:0.0128	train-error:0.000506
[191]	eval-error:0.0128	train-error:0.000506
[192]	eval-error:0.0128	train-error:0.000506
[193]	eval-error:0.0128	train-error:0.000506
[194]	eval-error:0.0128	train-error:0.000506
[195]	eval-error:0.0128	train-error:0.000506
[196]	eval-error:0.0128	train-error:0.000506
[197]	eval-error:0.0128	train-error:0.000506
[198]	eval-error:0.0128	train-error:0.000506
[199]	eval-error:0.0128	train-error:0.000506


In [27]:
def xgb_prediction(data, urls, bst, thresh = 0.5, ntree_limit = 0):
    data = xgb.DMatrix(data, [0 for _ in range(len(data))])
    prediction = []
    y_pred = bst.predict(data, ntree_limit = ntree_limit)
    for pred, url in zip(y_pred, urls):
        if pred > thresh:
            prediction.append(url)
    print "prediction:%d\trecall:%.3f" %(len(prediction), len(prediction)/float(len(urls)))
    return prediction

ntree_limit = -1
thresh = 0.8

pred_ben_train = xgb_prediction(train_ben_x, train_ben_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
pred_mal_train = xgb_prediction(train_mal_x, train_mal_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
print ""
pred_ben = xgb_prediction(test_ben_x, test_ben_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
pred_mal = xgb_prediction(test_mal_x, test_mal_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
print ""
pred_unk = xgb_prediction(unk_x, total_unk_urls, bst, thresh = thresh, ntree_limit = ntree_limit )

prediction:0	recall:0.000
prediction:8476	recall:0.996

prediction:13	recall:0.001
prediction:5166	recall:0.910

prediction:331	recall:0.225


In [28]:
pred_ben

['http://file.00809.cn/downgame/dkdw.exe',
 'http://cawxxa.mavinstationery.com/001/20180411/AVPlayer_185656.apk?attname=ssplayer.apk',
 'http://khit.cn/xldl.zip',
 'http://static.ilclock.com/gcld/updates/makecert.exe',
 'http://www-to--future-net-static.smartgslb.com/download/tofuture.apk?s=1',
 'http://soft3.xzstatic.com/1210/pptdjsjzh_DownZa.Cn.rar',
 'http://www.wdexam.com/mobile/app/wdexam.apk',
 'http://b612kaji-static.snowcam.cn/contents/sticker/60099/60099_3_30.zip',
 'http://tenby.720892.com/Install/yxendin_1.7z',
 'http://adwasd.xuezuocai88.com/001/20180417/AVPlayer_221528.apk?attname=ssplayer.apk',
 'http://app.1x948d.cn/homeman.apk',
 'http://www.wsb003.cn/LiveUpdate/DoctorRegManager/7.0.1.7/APP/DoctorRegManager.CBCL.dll.zip',
 'http://xy.shijialianzuiman.com/MNSY_91587044_0409.apk']

### extend data

In [ ]:
def get_untrack_data(total_domains, data_map):
    urls = []
    dd = set(total_domains) - set(data_map.keys())
    for url in total_urls:
        if urlparse.urlparse(url).hostname in dd:
            urls.append(url)
    return urls

In [ ]:
tst_url_ip_map = get_ip.make_url_ip_map(urls)
jl.dump(tst_url_ip_map,      '../data/exp_rule/tst/url_ip_map.jl.z')

In [40]:
tst_url_ip_dmap = get_ip.make_url_ip_dmap(urls)
jl.dump(tst_url_ip_dmap,     '../data/exp_rule/tst/url_ip_dmap.jl.z')

['../data/exp_rule/tst/url_ip_dmap.jl.z']

In [47]:
tst_domain_cert_map = get_cert.get_certificate_list(urls)
jl.dump(tst_domain_cert_map, '../data/exp_rule/tst/domain_cert_map.jl.z')

['../data/exp_rule/tst/domain_cert_map.jl.z']

In [ ]:
tst_domain_rank_map = get_rank.get_alexa_rank_list(urls, n_jobs=2)
jl.dump(tst_domain_rank_map, '../data/exp_rule/tst/domain_rank_map.jl.z')

In [ ]:
domain_pr_map = get_pagerank.get_pr_list(total_urls, pr_dict=domain_pr_map)
jl.dump(domain_pr_map, "../data/exp_rule/pr/pr.jl.z")

In [ ]:
domain_whois_map_update = get_whois.get_whois_list(total_urls, "../data/exp_rule/whois/", check=True, n_jobs=4)

In [84]:
sus_domain_update = get_sus_tld.extract_sus_url(total_urls)
sus_domain = sus_domain | sus_domain_update
jl.dump(sus_domain, "../data/exp_rule/sus_domain/sus_domain.jl.z")

domain cnt:9036


['../data/exp_rule/sus_domain/sus_domain.jl.z']